In [1]:
#thread;;

#require "core";;
#require "core.syntax";;
#require "stdio";;

open Printf;;
open Stdio;;
open Core;;

In [40]:
let executable = "/home/igandhi/Documents/build/bin/klee"

let klee_stats = "/home/igandhi/Documents/build/bin/klee-stats"

let klee_flags =
  [ "search=random-path"; "no-forking"; "track-instruction-time" ]

let stats_flags = [ "print-all"; "to-csv" ]

let directory = "../klee/examples/llvm-coreutils/bitcodes/"

let bitcode = "echo.bc"

(* number of paths to explore with klee *)
let path_n = 1

val executable : string = "/home/igandhi/Documents/build/bin/klee"


val klee_stats : string = "/home/igandhi/Documents/build/bin/klee-stats"


val klee_flags : string list =
  ["search=random-path"; "no-forking"; "track-instruction-time"]


val stats_flags : string list = ["print-all"; "to-csv"]


val directory : string = "../klee/examples/llvm-coreutils/bitcodes/"


val bitcode : string = "echo.bc"


val path_n : int = 1


In [41]:
exception FileError of string

let flags_to_str (flags : string list) : string =
  String.concat ~sep:" " @@ List.map flags (fun x -> "--" ^ x) 

(** run runs the given command, and returns each line of output of standard out*)
let run (cmd : string) (suppress : bool) : string list =
  let cmd = if suppress then cmd ^ " 2> /dev/null" else cmd in
  let inp = Unix.open_process_in @@ cmd in
  let r = In_channel.input_lines inp in
  In_channel.close inp;
  r
  
(* Outputs are logged as log1.csv, log2.csv... lowest_unused finds the lowest unused log number*)
let lowest_unused () : int = 
    let filenames = run ("ls logs/") false in
    let num_to_name n = "log" ^ string_of_int n ^ ".csv" in
    let is_unused n = not @@ List.mem filenames (num_to_name n) (String.equal) in
    match List.find (List.init 100 Fun.id) is_unused with
    | Some unused -> unused
    | None -> raise (FileError "All file numbers used")

let x = lowest_unused ()

let write_to_log (headers : string list) (values : string list) : unit =
  let i = lowest_unused () in
  let oc = Out_channel.create ~append:true @@ "logs/log" ^ string_of_int i ^ ".csv" in
  fprintf oc "%s\n" (Option.value (List.hd headers) ~default:"No headers");
  List.iter values (fprintf oc "%s\n");
  Out_channel.close oc

(** get_csv_stats returns the results of "klee-stats" as a tuple of (header, value) *)
let get_csv_stats (directory : string) : string * string =
  let command =
    klee_stats ^ " " ^ directory ^ "klee-last " ^ flags_to_str stats_flags
  in
  let csv_table = run command false in
  (Option.value (List.hd csv_table) ~default:"No header", 
  Option.value (List.hd @@ List.rev csv_table) ~default:"No rows")
  
let print_str_list = fun l -> List.iter l print_endline

exception FileError of string


val flags_to_str : string list -> string = <fun>


val run : string -> bool -> string list = <fun>


val lowest_unused : unit -> int = <fun>


val x : int = 3


val write_to_log : string list -> string list -> unit = <fun>


val get_csv_stats : string -> string * string = <fun>


val print_str_list : string list -> unit = <fun>


In [42]:
let clear_subdirs (directory : string) : unit = ( run ("rm -rf " ^ directory ^ "*/") false : string list) |> ignore

val clear_subdirs : string -> unit = <fun>


In [43]:
let main () =
  let klee_command =
    String.concat ~sep:" "
    @@ [ executable; flags_to_str klee_flags; directory ^ bitcode ]
  in
  let (h_str (* header strings *), v_str (* value strings *)) = Caml.List.split @@ Caml.List.init path_n @@ fun _ ->
      (run klee_command false : string list) |> ignore;
      get_csv_stats directory in
   print_endline @@ Caml.List.hd h_str;
   print_str_list v_str;
   clear_subdirs directory;
   write_to_log h_str v_str

let () = main ()

val main : unit -> unit = <fun>


KLEE: ERROR: Loading file ../klee/examples/llvm-coreutils/bitcodes/echo.bc failed: No such file or directory
No KLEE output directory found


No header
No rows
